### Install dependencies

In [1]:
!apt install swig cmake

Reading package lists... Done
Building dependency tree       
Reading state information... Done
swig is already the newest version (3.0.12-1).
cmake is already the newest version (3.10.2-1ubuntu2.18.04.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [2]:
!pip install -r https://huggingface.co/spaces/ThomasSimonini/temp-space-requirements/raw/main/requirements/requirements-unit1.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Create a virtual screen

In [3]:
!sudo apt-get update
!apt install python-opengl
!apt install ffmpeg
!apt install xvfb
!pip3 install pyvirtualdisplay

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 172 kB in 1s (159 kB/s)
Reading package lists... Don

In [4]:
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

### Imports

In [5]:
import gym

from huggingface_sb3 import load_from_hub, package_to_hub, push_to_hub
from huggingface_hub import (
    notebook_login,
)  # To log to our Hugging Face account to be able to upload models to the Hub.

from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_vec_env

### Training

In [6]:
# Create a vectorized environment for training
env = make_vec_env("BipedalWalker-v3", n_envs=16)

# Instantiate the agent
model = PPO(
    policy="MlpPolicy",
    env=env,
    n_steps=2048,
    batch_size=64,
    n_epochs=10,
    gamma=0.999,
    gae_lambda=0.95,
    ent_coef=0.001,
    verbose=1,
)

# Train the agent
model.learn(total_timesteps=int(3e6), progress_bar=True)

# Save the agent
model_name = "ppo_walker"
model.save(model_name)

Using cuda device


Output()

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 355      |
|    ep_rew_mean     | -112     |
| time/              |          |
|    fps             | 1292     |
|    iterations      | 1        |
|    time_elapsed    | 25       |
|    total_timesteps | 32768    |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 547        |
|    ep_rew_mean          | -112       |
| time/                   |            |
|    fps                  | 949        |
|    iterations           | 2          |
|    time_elapsed         | 69         |
|    total_timesteps      | 65536      |
| train/                  |            |
|    approx_kl            | 0.00585772 |
|    clip_fraction        | 0.0478     |
|    clip_range           | 0.2        |
|    entropy_loss         | -5.66      |
|    explained_variance   | -0.00454   |
|    learning_rate        | 0.0003     |
|   

### Evaluate

In [7]:
# Create a new environment for evaluation
eval_env = gym.make("BipedalWalker-v3")

# Evaluate the model
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)

# Print the results
print(f"mean_reward={mean_reward:.2f} +/- {std_reward:.2f}")

/usr/local/lib/python3.8/dist-packages/stable_baselines3/common/evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


mean_reward=296.78 +/- 1.20


### Push to Hub

In [8]:
notebook_login()

In [9]:
from stable_baselines3.common.vec_env import DummyVecEnv

env_id = 'BipedalWalker-v3'
model_architecture = 'PPO'
repo_id = "daspartho/ppo_walker"
commit_message = "Upload PPO BipedalWalker-v3 trained agent"
eval_env = DummyVecEnv([lambda: gym.make(env_id)])

package_to_hub(
    model=model,  # Our trained model
    model_name=model_name,  # The name of our trained model
    model_architecture=model_architecture,  # The model architecture we used: in our case PPO
    env_id=env_id,  # Name of the environment
    eval_env=eval_env,  # Evaluation Environment
    repo_id=repo_id,  # id of the model repository from the Hugging Face Hub
    commit_message=commit_message,
)

ℹ This function will save, evaluate, generate a video of your agent,
create a model card and push everything to the hub. It might take up to 1min.
This is a work in progress: if you encounter a bug, please open an issue.
Saving video to /tmp/tmp1n4t0c8i/-step-0-to-step-1000.mp4
ℹ Pushing repo daspartho/ppo_walker to the Hugging Face Hub
ℹ Your model is pushed to the Hub. You can view your model here:
https://huggingface.co/daspartho/ppo_walker/tree/main/


'https://huggingface.co/daspartho/ppo_walker/tree/main/'